# DP-DKI analysis from DDE MRI
### Hunter G. Moss (3/13/24)
#### Requires: FSL, MRTrix3 and PyDesigner, as well as Numpy, Shutil, and Nibabel Python packages

In [9]:
#---------------------------------------------------------------------
# Package Management
#---------------------------------------------------------------------
import sys as sys
import os # mkdir
import os.path as op # path
import subprocess
from shutil import copyfile
import numpy as np # array, ndarray
import numpy.matlib as npm
import nibabel as nib
subj = 'IAM_DEV_001' # subject name
root = op.join('/Volumes/Bindy/DPDKI',subj)  # main parent directory path the subject
dcmPath = op.join(root,'dicom') # obviously, a dicom folder path for the subject

dwiIn63dirs = op.join(dcmPath,'DPDKI_63dir_TE115_20') # Will need to change...
dwiIn9dirs_r1 = op.join(dcmPath,'DPDKI_9dir_TE115_b1000_run1_21') # Will need to change...
dwiIn9dirs_r2 = op.join(dcmPath,'DPDKI_9dir_TE115_b1000_run2_23') # Will need to change...
dwiIn39dirs_r1 = op.join(dcmPath,'DPDKI_39dir_TE115_b2000_run1_22') # Will need to change...
dwiIn39dirs_r2 = op.join(dcmPath,'DPDKI_39dir_TE115_b2000_run2_24') # Will need to change...

preprocessFlg = 0 # Tells the script to run PyDesigner (set to 0 afterwards)
calcFlg = 0

#############################################################################
####### SHOULD ONLY HAVE TO MESS WITH STUFF MINIMALLY BEYOND THIS POINT #####
#############################################################################

# FWHM Gaussian smoothing kernel (might need adjusting at 7T)
# after testing we found (1.5* voxel dimension) to be decent for 3T (might need adjusting at 7T)
fwhm = str(1.5) # it is a string bc it is passed to PyDesigner later on...

# Both Delta and tau are given by the machine I believe
# Needed for Exchane Rate estimate at the end...
Delta = 24.7 * 10**-3 # Diffusion time
# tau = 28.1 * 10**-3 # This was at TE = 115 ms (I think)
tau = (28.1 + 27) * 10**-3 # This was mixing time at TE = 140 ms (I think)

# We acquired two averages for b = 1000 and b = 2000 s/mm^2
# These are the b1000 run 1 and run 2
dpdkib1r1Dir = op.join(root,'DPDKI_b1_r1')
dpdkib1r2Dir = op.join(root,'DPDKI_b1_r2')

# These are the b2000 run 1 and run 2
dpdkib2r1Dir = op.join(root,'DPDKI_b2_r1')
dpdkib2r2Dir = op.join(root,'DPDKI_b2_r2')

# We also gathered a seperate DDE scan with 63 directions
dpdki63Dir = op.join(root,'DPDKI63')

# This is the final directory that will store all the processed data following 
# running PyDesigner (w/o the fitting routines, only pre-processing steps)
dpdkiDir = op.join(root,'DPDKI')

# File names and such (shouldn't need to change)
dpdki_all = op.join(dpdki63Dir,'dwi_preprocessed.nii')

b1_run1 = op.join(dpdkib1r1Dir,'dwi_preprocessed.nii')
b1_run2 = op.join(dpdkib1r2Dir,'dwi_preprocessed.nii')

b2_run1 = op.join(dpdkib2r1Dir,'dwi_preprocessed.nii')
b2_run2 = op.join(dpdkib2r2Dir,'dwi_preprocessed.nii')

dpdki_b1 = op.join(dpdkiDir,'dpdki_b1.nii')
dpdki_b2 = op.join(dpdkiDir,'dpdki_b2.nii')
dpdki63 = op.join(dpdkiDir,'dpdki63.nii')

# BEGIN - PRE-PROCESSING with PyDesigner....

if preprocessFlg == 1:
    
    os.makedirs(dpdki63Dir, exist_ok=True)

    dwiIn = dwiIn63dirs
    dwiDir = dpdki63Dir

    arg = [
        'pydesigner','-n',
        '-z','--fwhm',fwhm,
        '--mask','-cf',
        '-o',dwiDir,
        dwiIn,
        '--nofit',
        '--force'
        ]
    completion = subprocess.run(arg)

    os.makedirs(dpdkib1r1Dir, exist_ok=True)

    dwiIn = dwiIn9dirs_r1
    dwiDir = dpdkib1r1Dir

    arg = [
        'pydesigner','-n',
        '-z','--fwhm',fwhm,
        '--mask','-cf',
        '-o',dwiDir,
        dwiIn,
        '--nofit',
        '--force'
        ]
    completion = subprocess.run(arg)

    os.makedirs(dpdkib1r2Dir, exist_ok=True)

    dwiIn = dwiIn9dirs_r2
    dwiDir = dpdkib1r2Dir

    arg = [
        'pydesigner','-n',
        '-z','--fwhm',fwhm,
        '--mask','-cf',
        '-o',dwiDir,
        dwiIn,
        '--nofit',
        '--force'
        ]
    completion = subprocess.run(arg)

    os.makedirs(dpdkib2r1Dir, exist_ok=True)

    dwiIn = dwiIn39dirs_r1
    dwiDir = dpdkib2r1Dir

    arg = [
        'pydesigner','-n',
        '-z','--fwhm',fwhm,
        '--mask','-cf',
        '-o',dwiDir,
        dwiIn,
        '--nofit',
        '--force'
        ]
    completion = subprocess.run(arg)

    os.makedirs(dpdkib2r2Dir, exist_ok=True)

    dwiIn = dwiIn39dirs_r2
    dwiDir = dpdkib2r2Dir

    arg = [
        'pydesigner','-n',
        '-z','--fwhm',fwhm,
        '--mask','-cf',
        '-o',dwiDir,
        dwiIn,
        '--nofit',
        '--force'
        ]
    completion = subprocess.run(arg)

    os.makedirs(dpdkiDir, exist_ok=True)

    arg = [
        'mrcalc',
        b1_run1,
        b1_run2,
        '-add',
        '2','-div',
        dpdki_b1
    ]
    completion = subprocess.run(arg)

    arg = [
        'mrcalc',
        b2_run1,
        b2_run2,
        '-add',
        '2','-div',
        dpdki_b2
    ]
    completion = subprocess.run(arg)

    copyfile(dpdki_all,dpdki63)

###############################################################################################
# BEGIN - Calculations for DIFFUSIVITY and 4 KURTOSIS metrics of DP-DKI
###############################################################################################
if calcFlg == 1:
    # This just does erosion of the brain mask that comes out of PyDesigner
    arg = [
        'fslmaths',
        op.join(dpdki63Dir,'brain_mask.nii'),
        '-eroF',
        op.join(dpdkiDir,'brain_mask_eroF'),
        ]
    completion = subprocess.run(arg)

    brain_img = nib.load(op.join(dpdkiDir,'brain_mask_eroF.nii.gz'))
    mask = brain_img.get_fdata()

    # Uncommenting next two lines will load and also exclude CSF from calculations (assuming it is still output from PyDesigner)
    # csf_img = nib.load(op.join(dpdki63Dir,'csf_mask.nii')) 
    # mask[csf_img.get_fdata() == 1] = 0

    # HERE we begin loading in image data for the b1000, b2000 and the combo 63direction scheme data...
    dpdkib1_img = nib.load(dpdki_b1)
    dpdkib2_img = nib.load(dpdki_b2)
    dpdki63_img = nib.load(dpdki63)

    # Load in bvecs files
    bvecs63 = np.loadtxt(op.join(dpdki63Dir,'dwi_preprocessed.bvec'))
    bvecsb1 = np.loadtxt(op.join(dpdkib1r1Dir,'dwi_preprocessed.bvec'))
    bvecsb2 = np.loadtxt(op.join(dpdkib2r1Dir,'dwi_preprocessed.bvec'))

    # Load in bvals files
    bvals63 = np.rint(np.loadtxt(op.join(dpdki63Dir,'dwi_preprocessed.bval')))
    bvalsb1 = np.rint(np.loadtxt(op.join(dpdkib1r1Dir,'dwi_preprocessed.bval')))
    bvalsb2 = np.rint(np.loadtxt(op.join(dpdkib2r1Dir,'dwi_preprocessed.bval')))

    S0b1 = np.mean(dpdkib1_img.get_fdata()[:,:,:,bvalsb1 == 0],axis = 3) # B0 average from b1000 shell
    Sb1 = dpdkib1_img.get_fdata()[:,:,:,bvalsb1 != 0] # grab b1000 volumes (excluding b = 0 images)

    S0b2 = np.mean(dpdkib2_img.get_fdata()[:,:,:,bvalsb2 == 0],axis = 3) # B0 average from b2000 shell
    Sb2 = dpdkib2_img.get_fdata()[:,:,:,bvalsb2 != 0] # grab b2000 volumes (excluding b = 0 images)

    S063 = np.mean(dpdki63_img.get_fdata()[:,:,:,bvals63 == 0],axis = 3) # B0 average from 63dirs scheme
    # Grab the non-zero b-value images for b1000 and b2000 seperately from the 63 dirs scheme
    S63b1 = dpdki63_img.get_fdata()[:,:,:,bvals63 == 1000]
    S63b2 = dpdki63_img.get_fdata()[:,:,:,bvals63 == 2000]

    ###############################################################################################
    # BEGIN: rearranging and combining of data from the various schemes for later calculations
    ###############################################################################################
    psi0 = np.log((S0b1 + S0b2 + S063)/3)
    psi0[np.isnan(psi0)] = 0
    dims = np.shape(psi0)
    psi0 = np.reshape(psi0,np.prod(dims), order = 'F')

    Sb1 = (Sb1 + S63b1[:,:,:,0:9])/2
    Sb1[np.isnan(Sb1)] = 0
    Sb1 = np.reshape(Sb1,(np.prod(dims),9), order = 'F')

    Sb2 = (Sb2 + S63b2[:,:,:,0:39])/2
    Sb2[np.isnan(Sb2)] = 0
    Sb2 = np.reshape(Sb2,(np.prod(dims),39), order = 'F') 

    AmbL = np.log(Sb1)
    AmbH = np.log(Sb2)

    S63b1[np.isnan(S63b1)] = 0
    S63b1 = np.reshape(S63b1,(np.prod(dims),63), order = 'F')

    S63b2[np.isnan(S63b2)] = 0
    S63b2 = np.reshape(S63b2,(np.prod(dims),63), order = 'F')

    Am63b1 = np.log(S63b1)
    Am63b2 = np.log(S63b2)

    AmbL[np.isnan(AmbL)] = 0
    AmbH[np.isnan(AmbH)] = 0

    Am63b1[np.isnan(Am63b2)] = 0
    Am63b2[np.isnan(Am63b2)] = 0

    ###############################################################################################
    # END: rearranging and combining of data from the various schemes for later calculations
    ###############################################################################################

    ####################################################################################################################
    # BEGIN: using the above combo's strategically with Equations listed as seen in manuscript draft:
    # Moss, Benitez and Jensen - Simple Schemes for Estimating Rotationally Invariant Kurtosis Measures from DDE MRI
    ####################################################################################################################

    # Eq. 19 for b = 1000 (i.e., AmbL)
    psibL = (1/15) * np.sum(AmbL[:,0:3], axis = 1) + (2/15) * np.sum(AmbL[:,3:9],axis = 1)

    # Eq. 19 for b = 2000 (i.e., AmbH) 
    psibH = (1/15) * np.sum(AmbH[:,0:3], axis = 1) + (2/15) * np.sum(AmbH[:,3:9],axis = 1) 

    # Eq. 20 for b = 2000
    psibHt = (1/12) * np.sum(AmbH[:,3:15],axis = 1) + (1/24) * np.sum(AmbH[:,15:21],axis = 1) - (1/12) * np.sum(AmbH[:,0:3],axis = 1)
    psibHt[np.isnan(psibHt)] = 0
    psibHt[np.isinf(psibHt)] = 0

    # Eq. 21 for b = 2000
    dpsibHt = psibH - psibHt 
    dpsibHt[np.isnan(dpsibHt)] = 0
    dpsibHt[np.isinf(dpsibHt)] = 0

    # Eq.22 for b = 2000
    dpsibHAt = (2/5)*np.sum(AmbH[:,0:3],axis = 1) + (2/15)*np.sum(AmbH[:,3:9], axis = 1) - (2/15)*np.sum(AmbH[:,9:15], axis = 1) - (1/5)*np.sum(AmbH[:,15:21], axis = 1) - (1/15)*np.sum(AmbH[:,21:33], axis = 1) + (2/15)*np.sum(AmbH[:,33:39], axis = 1)
    dpsibHAt[np.isnan(dpsibHAt)] = 0
    dpsibHAt[np.isinf(dpsibHAt)] = 0

    # Eq. 36 b = 2000
    m = np.arange(16,22,1,dtype = 'int')

    psibHCt = (1/6)*np.sum((-1)**m * AmbH[:,15:21],axis = 1)
    psibHCt[np.isnan(psibHCt)] = 0
    psibHCt[np.isinf(psibHCt)] = 0

    Dbar = (3/2)*psi0 - 2*psibL + (1/2)*psibH #Eq. 31
    Wbar = (3/Dbar**2)*(psi0 - 2*psibL + psibH) #Eq. 32

    dWbar = (6/(Dbar**2 * 2**2)) * dpsibHt # Eq. 33
    dWAt = (6/(Dbar**2 * 2**2)) * dpsibHAt # Eq. 34

    Wbart = Wbar - dWbar
    WAt = Wbar - dWAt

    psibLt2 = (3/40)*np.sum(Am63b1[:,39:51],axis = 1) - (3/40)*np.sum(Am63b1[:,51:63],axis = 1) - (1/30)*np.sum((-1)**m * Am63b1[:,15:21],axis = 1)
    psibHt2 = (3/40)*np.sum(Am63b2[:,39:51],axis = 1) - (3/40)*np.sum(Am63b2[:,51:63],axis = 1) - (1/30)*np.sum((-1)**m * Am63b2[:,15:21],axis = 1)

    Cbar = ((1/2)*psibHt2 - psibLt2) # Eq. 37
    WBt = (12/(Dbar**2)) * ((1/2)*psibHt2 - psibLt2) # Eq. 38


    # We can now also calculate the associated anisotropies
    # First, we get the conventional FA
    G = (1/2) * ((AmbL[:,3] - AmbL[:,4])**2 + (AmbL[:,5] - AmbL[:,6])**2 + (AmbL[:,7] - AmbL[:,8])**2) # Eq. 42

    numer = ((AmbL[:,0] - AmbL[:,1])**2 + (AmbL[:,0] - AmbL[:,2])**2 + (AmbL[:,1] - AmbL[:,2])**2 + 3*G) # numerator of Eq. 41
    denom = ((psi0 - AmbL[:,0])**2 + (psi0 - AmbL[:,1])**2 + (psi0 - AmbL[:,2])**2 + G) # denominator of Eq. 41

    # The Fractional Anisotropy calculation
    fa = np.sqrt(numer/(2*denom)) # Eq. 41

    # Type-II microFA (Eq. 39)
    muFAprime = np.sqrt((30*dWbar)/(9 + 20*dWbar))

    # Type-I microFA (Eq. 40)
    muFA = np.sqrt(3 * (3*fa**2 + 3*muFAprime**2 - 4 * fa**2 * muFAprime**2)/(9 - 4 * fa**2 * muFAprime**2)) 

    # We can also calulate and estimate of the exchange rate 
    Re = (3/(2*Delta + 3*tau))*np.log(Wbar/WAt) # Eq. 54

    #############################################
    # END: Calculations of rotational invariants
    #############################################

    #####################################################################
    # BEGIN: Reshape arrays and write out the rotational invariant images
    #####################################################################

    Dbar = np.reshape(Dbar,(dims[0],dims[1],dims[2]),order = 'F')
    Wbar = np.reshape(Wbar,(dims[0],dims[1],dims[2]),order = 'F')
    Wbart = np.reshape(Wbart,(dims[0],dims[1],dims[2]),order = 'F')
    WAt = np.reshape(WAt,(dims[0],dims[1],dims[2]),order = 'F')
    Cbar = np.reshape(Cbar,(dims[0],dims[1],dims[2]),order = 'F')
    WBt = np.reshape(WBt,(dims[0],dims[1],dims[2]),order = 'F')
    fa = np.reshape(fa,(dims[0],dims[1],dims[2]),order = 'F')
    muFAprime = np.reshape(muFAprime,(dims[0],dims[1],dims[2]),order = 'F')
    muFA = np.reshape(muFA,(dims[0],dims[1],dims[2]),order = 'F')
    Re = np.reshape(Re,(dims[0],dims[1],dims[2]),order = 'F')

    # Here we zero out anything outside the brain using the prior mask calculation
    Dbar[mask == 0] = 0
    Wbar[mask == 0] = 0
    Wbart[mask == 0] = 0
    WAt[mask == 0] = 0
    Cbar[mask == 0] = 0
    WBt[mask == 0] = 0
    fa[mask == 0] = 0
    muFAprime[mask == 0] = 0
    muFA[mask == 0] = 0
    Re[mask == 0] = 0

    # Actually where the images are read out to disk....
    outname = op.join(dpdkiDir,'Dbar.nii')
    newimg = nib.Nifti1Image(Dbar, dpdki63_img.affine, dpdki63_img.header)
    nib.save(newimg, outname)

    outname = op.join(dpdkiDir,'Wbar.nii')
    newimg = nib.Nifti1Image(Wbar, dpdki63_img.affine, dpdki63_img.header)
    nib.save(newimg, outname)

    outname = op.join(dpdkiDir,'Wbart.nii')
    newimg = nib.Nifti1Image(Wbart, dpdki63_img.affine, dpdki63_img.header)
    nib.save(newimg, outname)

    outname = op.join(dpdkiDir,'WAt.nii')
    newimg = nib.Nifti1Image(WAt, dpdki63_img.affine, dpdki63_img.header)
    nib.save(newimg, outname)

    outname = op.join(dpdkiDir,'Cbar.nii')
    newimg = nib.Nifti1Image(Cbar, dpdki63_img.affine, dpdki63_img.header)
    nib.save(newimg, outname)
        
    outname = op.join(dpdkiDir,'WBt.nii')
    newimg = nib.Nifti1Image(WBt, dpdki63_img.affine, dpdki63_img.header)
    nib.save(newimg, outname)

    outname = op.join(dpdkiDir,'FA.nii')
    newimg = nib.Nifti1Image(fa, dpdki63_img.affine, dpdki63_img.header)
    nib.save(newimg, outname)

    outname = op.join(dpdkiDir,'muFAprime.nii')
    newimg = nib.Nifti1Image(muFAprime, dpdki63_img.affine, dpdki63_img.header)
    nib.save(newimg, outname)

    outname = op.join(dpdkiDir,'muFA.nii')
    newimg = nib.Nifti1Image(muFA,dpdki63_img.affine, dpdki63_img.header)
    nib.save(newimg, outname)

    outname = op.join(dpdkiDir,'Re.nii')
    newimg = nib.Nifti1Image(Re, dpdki63_img.affine, dpdki63_img.header)
    nib.save(newimg, outname)

    ################################################
    # END: Writing out rotational invariants to disk
    ################################################

## Make montage of B0, B1000 and B2000 both raw and denoised data

In [10]:
# subj = '' # subject name
# root = ' ' # main parent directory path the subject
dpdki63Dir = op.join(root,'DPDKI63')
rawImgFn = op.join(dpdki63Dir,'dwi_raw.nii')
deNImgFn = op.join(dpdki63Dir,'intermediate_nifti','1_dwi_denoised.nii')

bvals = np.rint(np.loadtxt(op.join(dpdki63Dir,'dwi_preprocessed.bval')))

rawImg = nib.load(rawImgFn)
deNImg = nib.load(deNImgFn)

b0raw = rawImg.get_fdata()[:,:,11,bvals == 0]
b0deN = deNImg.get_fdata()[:,:,11,bvals == 0]

b1raw = rawImg.get_fdata()[:,:,11,bvals == 1000]
b1deN = deNImg.get_fdata()[:,:,11,bvals == 1000]
 
b2raw = rawImg.get_fdata()[:,:,11,bvals == 2000]
b2deN = deNImg.get_fdata()[:,:,11,bvals == 2000]

outname = op.join(dpdki63Dir,'B0_ALL_raw_s11.nii')
imgOut = nib.Nifti1Image(b0raw, rawImg.affine, rawImg.header)
nib.save(imgOut, outname)

outname = op.join(dpdki63Dir,'B0_ALL_deN_s11.nii')
imgOut = nib.Nifti1Image(b0deN, rawImg.affine, rawImg.header)
nib.save(imgOut, outname)

outname = op.join(dpdki63Dir,'B1_ALL_raw_s11.nii')
imgOut = nib.Nifti1Image(b1raw, rawImg.affine, rawImg.header)
nib.save(imgOut, outname)

outname = op.join(dpdki63Dir,'B1_ALL_deN_s11.nii')
imgOut = nib.Nifti1Image(b1deN, rawImg.affine, rawImg.header)
nib.save(imgOut, outname)

outname = op.join(dpdki63Dir,'B2_ALL_raw_s11.nii')
imgOut = nib.Nifti1Image(b2raw, rawImg.affine, rawImg.header)
nib.save(imgOut, outname)

outname = op.join(dpdki63Dir,'B2_ALL_deN_s11.nii')
imgOut = nib.Nifti1Image(b2deN, rawImg.affine, rawImg.header)
nib.save(imgOut, outname)
